### Import the Librabries

In [30]:


import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests
import random


print('Libraries imported')

Libraries imported


###  Download and load Toronto Data

In [3]:
#Toronto Geographical Data on Neighbhorhoods/Boroughs
Tor_url='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050'
Tor_df = pd.read_html(Tor_url)
Tor_df_postcodes=Tor_df[0]
#print('imported dataframe has', Tor_df_postcodes['Postcode'].count(), "postcodes entries")
Tor_df_postcodes.head(10)
#df_postcodes.dtypes

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


In [20]:
Tor_df_postcodes[Tor_df_postcodes['Neighbourhood']=='Not Assigned'].count()

Postcode         0
Borough          0
Neighbourhood    0
dtype: int64

In [27]:
#Preprocessing
#Toronto - Get rid of the "Not assigned" / Replace

Tor_df_postcodes = Tor_df_postcodes[Tor_df_postcodes["Borough"] != "Not assigned"]

#Replace the N/A values of Neighborhood with the Borough Value
#Tor_df_postcodes["Neighbourhood"].replace("Not assigned", Tor_df_postcodes["Borough"], inplace=True)

Tor_df_postcodes.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [28]:
#Now combine all the neighbourhoods with the same postal codes
Tor_df = Tor_df_postcodes.groupby(["Postcode","Borough"])["Neighbourhood"].apply(list)
Tor_df = Tor_df.sample(frac=1).reset_index()
Tor_df["Neighbourhood"] = Tor_df["Neighbourhood"].str.join(', ')
Tor_df.head(10)

,Postcode,Borough,Neighbourhood
0,M9W,Etobicoke,Northwest
1,M1W,Scarborough,L'Amoreaux West
2,M1B,Scarborough,"Rouge, Malvern"
3,M2P,North York,York Mills West
4,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
5,M3M,North York,Downsview Central
6,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade
7,M2R,North York,Willowdale West
8,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."
9,M4G,East York,Leaside


In [29]:
Tor_df.shape

(103, 3)

In [31]:
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [32]:
#Get the Latitude and Langitude to match the Postcode
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
geo = pd.read_csv("Geospatial_Coordinates.csv", index_col = 0)         
Tor_df = Tor_df.join(geo, on = "Postcode")

#Take only those neighborhoods that contain the word Toronto
Tor_df = Tor_df[Tor_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
Tor_df.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
1,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
2,M4S,Central Toronto,Davisville,43.704324,-79.388790
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M4P,Central Toronto,Davisville North,43.712751,-79.390197
